In [1]:
%env PGSERVER=pgdb-plastico-dev.postgres.database.azure.com
%env PGDATABASE=plastico-dev
%env PGUSERNAME=writer_user@pgdb-plastico-dev
%env PGPWD=SurfWriter!

env: PGSERVER=pgdb-plastico-dev.postgres.database.azure.com
env: PGDATABASE=plastico-dev
env: PGUSERNAME=writer_user@pgdb-plastico-dev
env: PGPWD=SurfWriter!


In [2]:
# import prerequesite for blob
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient
from utils.blob import list_blob_in_container,get_blob_infos,download_blob
# import prerequesite for ai
import json
import requests
import logging
from utils.ai import is_ai_ready,get_prediction,get_json_prediction,get_clean_timed_prediction,get_trash_label,map_label_to_trash_id_PG,get_trash_first_time,get_trash_time_index,get_df_prediction,get_df_manual_trash
# import prerequesite for gps
import gpxpy
import gpxpy.gpx
import json
import subprocess
import datetime
from datetime import datetime
from datetime import timedelta
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm
from utils.gps import extract_gpx_from_gopro,parse_gpx,get_gps_point_list,create_time,create_latitude,create_longitude,create_elevation,fill_gps,long_lat_to_shape_point,transform_geo,get_df_trash_gps,get_gpx_name,get_df_manual_gps
# import prerequesite from media
from utils.media import get_media_duration,get_media_fps
# import prerequesite for postgre
import os
import psycopg2
from utils.postgre import get_pg_connection_string,open_pg_connection,close_pg_connection,insert_trash_2,insert_trash_df,get_df_data
# import exception
from utils.exceptions import ETLError
import warnings
warnings.filterwarnings('ignore')
# import argparse to pass parameters to main function
import argparse
import pathlib

In [ ]:
with open('../data/prediction.json') as json_file:
    jsonPrediction = json.load(json_file)

In [3]:
# OSM Tracker GPX with Trash - no AI is required
gpx_path = '../data/ADOUR.1_AVRIL_2019.ANTOINE_G1.gpx'
gpx_data = parse_gpx(gpx_path)
gps_points = gpx_data.waypoints

In [ ]:
import pandas as pd
def get_df_manual_gps(gpx_data_waypoints):
    gps_list = []
    for waypoint in gpx_data_waypoints:
        gps_point = {'Time': waypoint.time, 'Latitude': waypoint.latitude,
                              'Longitude': waypoint.longitude, 'Elevation': waypoint.elevation}
        shape_gps_point = long_lat_to_shape_point(gps_point)
        geo_2154 = transform_geo(shape_gps_point)
        geo_2154_gps_point = {'Time': shape_gps_point['Time'],'the_geom':geo_2154, 'Latitude':shape_gps_point['Latitude'],'Longitude': shape_gps_point['Longitude'], 'Elevation':shape_gps_point['Elevation']}
        gps_list.append(geo_2154_gps_point)
    df_manual_gps = pd.DataFrame(gps_list)
    return df_manual_gps

In [ ]:
df = get_df_manual_gps(gpx_data.waypoints)

In [ ]:
df_manual_gps = get_df_manual_gps(gpx_data.waypoints)
df_manual_trash = get_df_manual_trash(gpx_data.waypoints)
df_data = get_df_data(df_manual_trash,df_manual_gps)

In [ ]:
def get_df_manual_trash(gpx_data_waypoints):
    trash_list = []
    i = 0
    for waypoint in gpx_data_waypoints:
        trash_type_id = map_label_to_trash_id_PG(waypoint.name.lower())
        trash = {'id':i,'label':waypoint.name,'trash_type_id':trash_type_id}
        trash_list.append(trash)
        i = i+1
    df_manual_trash = pd.DataFrame(trash_list)
    return df_manual_trash

In [ ]:
df = get_df_manual_trash(gpx_data.waypoints)

In [ ]:
video_name = '28022020_Boudigau_4.MP4'
prediction = jsonPrediction['detected_trash'][0]
media_fps = get_media_fps(f'/tmp/{video_name}')
gpx_path = extract_gpx_from_gopro(f'/tmp/{video_name}')
gpx_data = parse_gpx(gpx_path)
gps_points = get_gps_point_list(gpx_data)
video_duration = get_media_duration(f'/tmp/{video_name}')
video_duration_sup = int(video_duration)+1
gps_points_filled = fill_gps(gps_points,video_duration_sup)
time_index = get_trash_time_index(prediction,media_fps)
# trash_gps
trash_gps = gps_points_filled[time_index]
shape_trash_gps = long_lat_to_shape_point(trash_gps)
geo_2154 = transform_geo(shape_trash_gps)
geo_2154_trash_gps = {'Time': shape_trash_gps['Time'], 'the_geom': geo_2154,'Latitude':shape_trash_gps['Latitude'],'Longitude':shape_trash_gps['Longitude'], 'Elevation': shape_trash_gps['Elevation']}
# get trash_type from ai module
label = get_trash_label(prediction)
trash_type = map_label_to_trash_id_PG(label)

In [4]:
DOWNLOAD_PATH = '/tmp'
AI_PORT = '5000'

In [5]:
pgserver = os.getenv("PGSERVER")
pgdatabase = os.getenv("PGDATABASE")
pgusername = os.getenv("PGUSERNAME")
pgpassword = os.getenv("PGPWD")

In [6]:
blob_name = 'ADOUR.1_AVRIL_2019.ANTOINE_G1.gpx'

In [7]:
blob_gpx_name = get_gpx_name(blob_name)
if not blob_gpx_name in os.listdir(DOWNLOAD_PATH):
    blob_gpx_client = BlobClient.from_connection_string(conn_str=connection_string,            container_name=campaign_container_name, blob_name=blob_gpx_name)
    download_blob(blob_gpx_client,DOWNLOAD_PATH)
gpx_path = f'{DOWNLOAD_PATH}/{blob_gpx_name}'
gpx_data = parse_gpx(gpx_path)
gps_points = gpx_data.waypoints
df_manual_gps = get_df_manual_gps(gps_points)
df_manual_trash = get_df_manual_trash(gps_points)
df_data = get_df_data(df_manual_trash,df_manual_gps)

In [8]:
df_data.head()

,id,label,trash_type_id,Time,the_geom,Latitude,Longitude,Elevation
0,0,Autre dechet,0,2019-04-01 13:22:06+00:00,POINT (463575.5239753085 6235764.194847712),43.181938,0.093354,394.293762
1,1,Autre dechet,0,2019-04-01 13:22:07+00:00,POINT (463575.0244292183 6235764.800646954),43.181944,0.093347,392.796753
2,2,Autres dechets +10,8,2019-04-01 13:22:50+00:00,POINT (463604.6561084815 6235794.644867932),43.182222,0.093698,387.526794
3,3,Autre dechet,0,2019-04-01 13:23:28+00:00,POINT (463640.9375533302 6235825.846384096),43.182514,0.094129,396.564392
4,4,Autre dechet,0,2019-04-01 13:23:30+00:00,POINT (463641.0869263107 6235826.829900276),43.182523,0.094131,396.978882


In [9]:
pg_conn_string = get_pg_connection_string()
pg_connection = open_pg_connection(pg_conn_string)
pg_cursor = pg_connection.cursor()
# Store row_id when insert
row_id_list = []

In [10]:
import logging
logger = logging.getLogger()
for i,row in tqdm(df_data.iterrows()):
    try:
        row_id = insert_trash_df(row,pg_cursor,pg_connection)
        logger.info(row_id)
        row_id_list.append(row_id)
    except:
        prediction_id = row['id']
        logger.error(f'There was an issue inserting Trash id: {prediction_id} within PostGre')
        logger.error("Early exit of ETL workflow as PG INSERT failed")
        exit()

logger.info(f'Successfully inserted {str(len(row_id_list))} Trashes within Trash table')    

# Close PG connection
close_pg_connection(pg_connection)

90it [00:37,  2.42it/s]
